<a href="https://colab.research.google.com/github/cvs2010/udemyDeepLearning/blob/main/Projeto_5_scraping_data_zapimoveis_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas

In [82]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd
import re
import requests
import math
from numpy.lib.function_base import quantile

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.11.2
urllib -> 3.10
pandas -> 1.5.3


# Webscrapping de imóveis a venda em Zapimoveis

In [33]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(500)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = "https://www.zapimoveis.com.br/venda/?pagina="+str(i)+"&tipo=Im%C3%B3vel%20usado&transacao=Venda"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

        #tipo    
        if re.search("kitnet|kitnets|kit|kit net|kitinete|kitinetes|quitinete|kitão|germinada|geminada|conjugado", card['descricao'].lower()):    
            card['tipo'] =  int(1)
        elif re.search("apartamento|apartamentos|apto|cobertura", card['descricao'].lower()):
            card['tipo'] =  int(2)
        elif re.search("casa de condominio|casa de condomínio|casa de condominios|casa de condomínios", card['descricao'].lower()):
            card['tipo'] =  int(3)    
        elif re.search("studio|studios", card['descricao'].lower()):
            card['tipo'] =  int(4)
        elif re.search("vila|villa", card['descricao'].lower()):    
            card['tipo'] =  int(5)
        elif re.search("casa|casas", card['descricao'].lower()):
            card['tipo'] =  int(6)    
        elif re.search("loft|duplex|dupllex", card['descricao'].lower()):    
            card['tipo'] =  int(7)
        elif re.search("loteamento|lote|terrenos|terreno|trrno|lote/terreno", card['descricao'].lower()):    
            card['tipo'] =  int(8)
        else:
            card['tipo'] =  int(9)
                
       #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)  
            
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
                        
        cards.append(card)

        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-imovel-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
tipo             0
suite            0
academia         0
varanda          0
transporte       0
salao            0
planejado        0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,tipo,suite,academia,varanda,transporte,salao,planejado,acabamento
0,1600000,Venha morar em um resort exclusivo com mais de...,Avenida Professor Florestan Fernandes,Camboinhas,4,2,3,140,537,1761,Super destaque,2,0,0,0,0,0,0,0
1,0,Rua Professor Gabizo (Tijuca). Oportunidade!!!...,Rua Professor Gabizo,Tijuca,2,0,2,150,1319,600,Super destaque,2,0,0,0,0,0,0,0
2,1299900,"Casa para Venda no bairro Vila Pompéia, locali...",Pompeia,São Paulo,3,2,3,220,1070,0,Super destaque,5,1,0,0,0,0,0,0
3,710000,Terras alphaville Sergipe 2 Casa nova em Alph...,,Barra dos Coqueiros,3,0,1,150,0,0,Nenhuma,6,1,0,0,0,0,0,0
4,695000,"Imóvel muito bem localizado, em uma das melhor...",Rua Antônio Bertoni Garcia,Jardim Antonio Von Zuben,1,4,3,142,2164,0,Nenhuma,9,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,500000,Vazio,Travessa Dona Ana,Vila São Paulo,3,2,2,200,0,0,Nenhuma,9,0,0,0,0,0,0,0
9996,2660000,Vazio,Rua Monte Alegre,Perdizes,4,3,2,350,1300,0,Nenhuma,9,0,0,0,0,0,0,0
9997,2300000,Vazio,Rua Mapuá,Chácara Monte Alegre,4,3,8,411,1614,0,Nenhuma,9,0,0,0,0,0,0,0
9998,1200000,Vazio,Rua Embuaçu,Vila Mariana,2,2,1,160,570,0,Nenhuma,9,0,0,0,0,0,0,0


# Webscrapping de casas em Zapimoveis

In [35]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(500)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason) 

    
    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
        
        #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)  
        
        
        #acabamento
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
         
        cards.append(card)


        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-casas-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
suite            0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,acabamento
0,447000,Uma casa muito charmosa e em uma ótima localiz...,Morretes,Itapema,2,1,1,62,300,10,Super destaque,0,0
1,380000,"DORMITÓRIOS AMPLOS, CASA COM PISO FRIO, SALA P...",Palmeiras de São José,São José dos Campos,2,2,1,70,0,0,Super destaque,0,0
2,48000000,"Belíssima casa com projeto de Antônio Scarpa, ...",Rua Marechal Bitencourt,Jardim Paulista,4,10,6,1174,12900,0,Super destaque,0,0
3,290000,A casa está localizado no bairro Jardim Piraju...,Rua Desembargador Alceu Cordeiro Fernandes,Jardim Pirajussara,3,2,3,130,1500,0,Nenhuma,1,0
4,320000,Entrada de 10% + Parcelas Fixas Facilidade par...,Rua Matilde Moreno,Jardim Ipanema,3,3,3,130,1800,0,Destaque,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,"Casa de 5 quartos, sendo 1 suíter master e 2 s...",Avenida Novara,Bandeirantes,5,8,6,467,908,0,Nenhuma,1,0
9996,950000,A Vila Alta Premium apresenta casa em condomín...,Rua General Álvaro Braga,Pilarzinho,3,2,4,176,0,0,Destaque,0,0
9997,580000,A casa no bairro Vinhais possui 150 metros qua...,Rua do Arame,Vinhais,4,2,3,150,0,0,Nenhuma,1,0
9998,550000,Ótimo imóvel comercial no centro de Cotia. Imó...,Centro,Cotia,3,2,2,170,190,0,Nenhuma,1,1


# Web Scraping de apartamentos do Zapimoveis

In [ ]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(500)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
         #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)    
                        
        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-ap-geral.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

# Web Scraping de imóveis no DF do Zapimoveis

In [95]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site
for i in range(pagestimes):
    # Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/imoveis/df+brasilia/?pagina=' + str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    try:
        response = requests.get(response, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print("Erro na requisição:", e)

    # Obtendo a quantidade de imóveis
    quantidade_imoveis_tag = soup.find('h1', {'class': 'summary__title js-summary-title heading-regular heading-regular__bold align-left text-margin-zero results__title'})
    if quantidade_imoveis_tag is not None:
        quantidade_imoveis_text = quantidade_imoveis_tag.find('strong').get_text()
        quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
        quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = "https://www.zapimoveis.com.br/venda/imoveis/df+brasilia/?pagina="+str(i)+"&tipo=Im%C3%B3vel%20usado&transacao=Venda"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)
        

        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

        #tipo    
        if re.search("kitnet|kitnets|kit|kit net|kitinete|kitinetes|quitinete|kitão|germinada|geminada|conjugado", card['descricao'].lower()):    
            card['tipo'] =  int(1)
        elif re.search("apartamento|apartamentos|apto|cobertura", card['descricao'].lower()):
            card['tipo'] =  int(2)
        elif re.search("casa de condominio|casa de condomínio|casa de condominios|casa de condomínios", card['descricao'].lower()):
            card['tipo'] =  int(3)    
        elif re.search("studio|studios", card['descricao'].lower()):
            card['tipo'] =  int(4)
        elif re.search("vila|villa", card['descricao'].lower()):    
            card['tipo'] =  int(5)
        elif re.search("casa|casas", card['descricao'].lower()):
            card['tipo'] =  int(6)    
        elif re.search("loft|duplex|dupllex", card['descricao'].lower()):    
            card['tipo'] =  int(7)
        elif re.search("loteamento|lote|terrenos|terreno|trrno|lote/terreno", card['descricao'].lower()):    
            card['tipo'] =  int(8)
        else:
            card['tipo'] =  int(9)
                
       #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)  
            
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
                        
        cards.append(card)

        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-imovel-df.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
400 Bad Request
400 Bad Request
308 Permanent Redirect
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
308 Permanent Redirect
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
308 Permanent Redirect
308 Permanent Redirect
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
308 Permanent Redirect
308 Permanent Redirect
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Request
400 Bad Re

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,tipo,suite,academia,varanda,transporte,salao,planejado,acabamento
0,0,"Viva próximo de tudo o que você deseja, na mel...",Rua 3,Norte,2,1,2,65,0,0,Na planta,2,0,0,0,1,0,0,0
1,0,"Viva próximo de tudo o que você deseja, na mel...",Rua 3,Norte,3,1,2,83,0,0,Na planta,2,0,0,0,1,0,0,0
2,0,O Reserva Parque Clube é assim: a combinação p...,EQS 414/415,Asa Sul,"[2, 3]",0,1,67,0,0,Em construção,9,0,0,0,0,0,0,0
3,0,"Vendo Excelente prédio, 04 apartamentos e 02 l...",Setor SIG Conjunto A,Taguatinga Norte,1,0,1,534,0,0,Super destaque,2,0,0,0,0,0,0,0
4,0,MODERN LIFE 1 QUARTO 40 M² MOBILIADO!!! A ...,Rua 5,Norte,1,0,1,40,795,329,Super destaque,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25095,0,CORUMBÁ IV LOTE NA BEIRA DO LAGO * CONDOMÍNI...,Condomínio Privê I Quadra 1 Conjunto D,Setor de Mansoes do Lago Norte,1,0,1,1000,0,0,Nenhuma,8,0,0,0,0,0,0,0
25096,0,rreno/Lote Corumbá IV. CORUMBÁ IV LOTE NA BE...,Condomínio Privê I Quadra 1 Conjunto E,Setor de Mansoes do Lago Norte,1,0,1,1000,0,0,Nenhuma,8,0,0,0,0,0,0,0
25097,0,", (Corumbá 4). Corumbá IV, Perfeito, Mensais ...",Quadra 1 Conjunto 2 Lote 1 Bloco C,Paranoa Parque,1,0,1,1000,0,0,Nenhuma,8,0,0,0,0,0,0,0
25098,0,"rumbá IV, /Corumba 4. /Corumba 04, Mega Lança...",Quadra 3 Conjunto A Bloco I,Taguatinga Norte,1,0,1,1000,0,0,Nenhuma,8,0,0,0,0,0,0,0


# Web Scraping de casas no DF do Zapimoveis

In [94]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site
for i in range(pagestimes):
    # Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/casas/df+brasilia/?pagina=' + str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    try:
        response = requests.get(response, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print("Erro na requisição:", e)

    # Obtendo a quantidade de imóveis
    quantidade_imoveis_tag = soup.find('h1', {'class': 'summary__title js-summary-title heading-regular heading-regular__bold align-left text-margin-zero results__title'})
    if quantidade_imoveis_tag is not None:
        quantidade_imoveis_text = quantidade_imoveis_tag.find('strong').get_text()
        quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
        quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    
    ##Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/casas/df+brasilia/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='simple-card__box')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quarto-min'], card['quarto-max'] = '1','1' 
        elif '-' in quartos.get_text() :
            card['quarto-min'], card['quarto-max'], *outro = quartos.get_text().replace(' ','').replace('\n','').split('-')
        else:        
           card['quarto-min'], card['quarto-max'] = quartos.get_text().replace(' ','').replace('\n',''),quartos.get_text().replace(' ','').replace('\n','')    
        
        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro-min'], card['banheiro-max'] = '1','1'
        elif '-' in banheiro.get_text() :
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n',''),banheiro.get_text().replace(' ','').replace('\n','')

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)-min'], card['area(m2)-max'] = '10','30'
        elif '-' in metro.get_text() :
            card['area(m2)-min'], card['area(m2)-max'], *outro = metro.get_text().replace(' ','').replace('\n','').replace('m²','').split('-')
        else:
            card['area(m2)-min'], card['area(m2)-max'] = metro.get_text().replace(' ','').replace('\n','').replace('m²',''),metro.get_text().replace(' ','').replace('\n','').replace('m²','')

            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)  
        
        
        #acabamento
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
         
        cards.append(card)

        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-casas-df.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
valor            0
descricao        0
local            0
bairro/cidade    0
quarto-min       0
quarto-max       0
garagem          0
banheiro-min     0
banheiro-max     0
area(m2)-min     0
area(m2)-max     0
iptu             0
condominio       0
novidade         0
suite            0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quarto-min,quarto-max,garagem,banheiro-min,banheiro-max,area(m2)-min,area(m2)-max,iptu,condominio,novidade,suite,acabamento
0,610000,QR 406 C Casa Belíssima com três pavimentos ...,QR 406 Conjunto 26,Samambaia Norte,4,4,2,2,2,300,300,0,0,Nenhuma,1,0
1,1200000,"Casa à Venda, 420m² por R$ 1.200.000. COLÔNIA ...",EQS 414/415,Asa Sul,3,3,2,4,4,420,420,0,0,Nenhuma,0,0
2,2200000,"SMT 22 Casa Com 4 Dormitórios à Venda, 380 m...",Setor de Mansões Taguatinga,Taguatinga Sul,4,4,0,4,4,380,380,0,0,Nenhuma,0,0
3,116800,Lotes a prestação próximo ao shopping sul do V...,Condomínio Porto Rico,Santa Maria,2,2,0,2,2,300,300,0,0,Nenhuma,0,0
4,55000,Casa com 02 quartos com opção para fazer o 3° ...,QR 425,Samambaia Norte,2,2,3,2,2,126,126,220,0,Nenhuma,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1299000,"Casa à Venda, 230m² por R$ 1.299.000. Setor Ha...",Setor Habitacional Arniqueira,Brasília,3,3,2,4,4,230,230,0,0,Nenhuma,0,0
7996,1600000,Oportunidade!!! Linda casa em condomínio fecha...,Rua 8 Chácara 200,Setor Habitacional Vicente Pires,4,4,4,4,4,420,420,0,0,Nenhuma,1,0
7997,2100000,(OZIEL LISBOA CORRETOR) VENDO CASA ALTO PADRÃ...,Rua 5,Setor Habitacional Vicente Pires,3,3,5,5,5,600,600,0,175,Nenhuma,0,0
7998,980000,"Casa à Venda, 400m² por R$ 980.000. Setor Habi...",Condomínio Belvedere Green,Setor Habitacional Jardim Botânico,5,5,0,3,3,400,400,0,0,Nenhuma,0,0


# Web Scraping de apartamentos no DF do Zapimoveis

In [91]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site
for i in range(pagestimes):
    # Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/df+brasilia/?pagina=' + str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    try:
        response = requests.get(response, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print("Erro na requisição:", e)

    # Obtendo a quantidade de imóveis
    quantidade_imoveis_tag = soup.find('h1', {'class': 'summary__title js-summary-title heading-regular heading-regular__bold align-left text-margin-zero results__title'})
    if quantidade_imoveis_tag is not None:
        quantidade_imoveis_text = quantidade_imoveis_tag.find('strong').get_text()
        quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
        quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/df+brasilia/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)
        

        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
         #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)    
                        
        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-ap-df.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

308 Permanent Redirect
308 Permanent Redirect
308 Permanent Redirect
valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
suite            0
academia         0
varanda          0
transporte       0
salao            0
planejado        0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,academia,varanda,transporte,salao,planejado
0,0,"Viva próximo de tudo o que você deseja, na mel...",Rua 3,Norte,2,1,2,65,0,0,Na planta,0,0,0,1,0,0
1,0,"Viva próximo de tudo o que você deseja, na mel...",Rua 3,Norte,3,1,2,83,0,0,Na planta,0,0,0,1,0,0
2,0,O Reserva Parque Clube é assim: a combinação p...,EQS 414/415,Asa Sul,"[2, 3]",0,1,67,0,0,Em construção,0,0,0,0,0,0
3,0,VENDIDO!! GRANDE OPORTUNIDADE! Ótimo Apartam...,Jardins Mangueiral,Brasília,2,1,1,48,0,0,Super destaque,0,0,1,0,0,0
4,0,Tetto Empreendimentos Imobiliarios Creci 13579...,Rua 3,Norte (Águas Claras),2,1,3,77,1250,850,Super destaque,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,0,DALL´OCA IMÓVEIS VENDE: Quer morar ou investi...,SHIN QI 1,Setor de Habitações Individuais Norte,1,1,2,64,0,730,Nenhuma,0,0,0,0,0,0
5696,0,"Neo Residencia Clnw, 10/11 Noroeste APARTAMEN...",CLNW 10/11 Lote C,Setor Noroeste,1,1,1,32,0,360,Nenhuma,0,0,0,0,0,0
5697,0,SQNW 110 FACILIDADE DE ACESSOS PRÓXIMO A COLÉG...,SQNW 110 Bloco J,Setor Noroeste,4,3,4,194,0,0,Destaque,0,0,0,0,0,0
5698,0,Excelente apartamento de dois quartos sendo um...,CCSW 4 Lote 1 Bloco B,Setor Sudoeste,2,1,2,66,0,508,Nenhuma,1,0,1,0,0,0
